# Historical Constituent of S&P500

Using Python by web scraping Wikipedia as it provides more historical data

In [7]:
import pandas as pd
import requests
from datetime import datetime
import numpy as np

In [10]:
headers = {
    'Accept': '*/*',
    'Accept-Language': 'en-US,en;q=0.8',
    'Cache-Control': 'max-age=0',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.116 Safari/537.36',
    'Connection': 'keep-alive',
    'Referer': 'http://www.baidu.com/'
}

url = 'https://en.wikipedia.org/wiki/List_of_S&P_500_companies'
html = requests.get(url, headers=headers).content

In [ ]:
html

In [11]:
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S&P_500_companies')

URLError: &lt;urlopen error [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。&gt;

In [2]:
data

NameError: name &#39;data&#39; is not defined